In [1]:
import rasterio
from rasterio.enums import Resampling
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('ggplot')


### Resizing and saving PRS imgs and GT (Chl-a maps)

In [6]:
# Open the input file
workdir="./data/inputs/intersected_inputs/"

list_ids = [1,2,4,6,10,13,17,18,19,21,23,24]

OUT_PATH_PRS = workdir+"PRS_imgs_300/"
OUT_PATH_GT = workdir+"GT_30_no_anomalies/"

for idx in list_ids:
        
    with rasterio.open(workdir+f"PRS_imgs/{str(idx)}.tif") as src:

        # Get the current metadata
        meta = src.meta

        # Update the metadata to reflect the new resolution
        meta.update({
            'transform': rasterio.Affine(meta['transform'].a * 10, meta['transform'].b, meta['transform'].c, 
                                         meta['transform'].d, meta['transform'].e * 10, meta['transform'].f),
            'width': src.width // 10,
            'height': src.height // 10,
            'res': (meta['transform'].a * 10, meta['transform'].e * 10)
        })

        # Create the output file
        with rasterio.open(OUT_PATH_PRS+f"{str(idx)}.tif", "w", **meta) as dst:

            # Resample the data for each band
            for i in range(1, src.count+1):
                band = src.read(i, out_shape=(meta['height'], meta['width']), resampling=Resampling.nearest)
                dst.write(band, i)
       
    ### GT_continuous
    # Open the input file
    with rasterio.open(workdir+f"GT_300_no_anomalies/{str(idx)}.tif") as src:

        # Get the current metadata
        meta = src.meta

        # Update the metadata to reflect the new resolution
        meta.update({
            'transform': rasterio.Affine(meta['transform'].a / 10, meta['transform'].b, meta['transform'].c, 
                                         meta['transform'].d, meta['transform'].e / 10, meta['transform'].f),
            'width': src.width * 10,
            'height': src.height * 10,
            'res': (meta['transform'].a / 10, meta['transform'].e / 10)
        })

        # Create the output file
        with rasterio.open(OUT_PATH_GT+f"{str(idx)}.tif", "w", **meta) as dst:

            # Resample the data using the nearest neighbor method
            data = src.read(
                out_shape=(src.count, meta['height'], meta['width']),
                resampling=Resampling.nearest
            )

            # Write the data to the output file
            dst.write(data)